In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002C32B5CBE00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C32B770C20>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Anay and I am a Agentic AI learner")])

AIMessage(content='Hello Anay! 👋🏼 It’s great to meet an Agentic AI learner. How can I support you today? Are you working on a specific project, exploring a particular framework, or just looking for general guidance on building autonomous agents? Let me know what you’re interested in, and we can dive right in!', additional_kwargs={'reasoning_content': 'User says "Hi, My name is Anay and I am a Agentic AI learner". They are likely greeting. We need to respond politely, maybe ask about their goals. Provide helpful info.'}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 86, 'total_tokens': 201, 'completion_time': 0.117112923, 'completion_tokens_details': {'reasoning_tokens': 41}, 'prompt_time': 0.004084915, 'prompt_tokens_details': None, 'queue_time': 0.156226965, 'total_time': 0.121197838}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_a8c584dda7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Anay and I am a Agentic AI learner"),
        AIMessage(content="Hello Anay! 👋🏼 It’s great to meet an Agentic AI learner. How can I support you today? Are you working on a specific project, exploring a particular framework, or just looking for general guidance on building autonomous agents? Let me know what you’re interested in, and we can dive right in!"),
        HumanMessage(content="Hey Whats my name and what do i do?")
        
    ]
)

AIMessage(content='You’re Anay, and you’re learning to build and work with Agentic AI systems—autonomous agents that can plan, act, and adapt to achieve goals. 🚀 If you’d like to dive deeper into any specific aspect (like architecture, tools, or project ideas), just let me know!', additional_kwargs={'reasoning_content': 'User says: "Hey Whats my name and what do i do?" They previously said: "Hi, My name is Anay and I am a Agentic AI learner". So we should respond with name and what they do: Anay, Agentic AI learner. Also maybe ask more.'}, response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 171, 'total_tokens': 302, 'completion_time': 0.147937799, 'completion_tokens_details': {'reasoning_tokens': 60}, 'prompt_time': 0.009200233, 'prompt_tokens_details': None, 'queue_time': 0.047651907, 'total_time': 0.157138032}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'm

### Message History
We can use a Message Histroy class to wrap our model and make it stateful. This will keep track of outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [11]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Anay and I am an Agentic AI learner")
    ],
    config=config
)

In [14]:
response.content

'Hi Anay! 👋 I’m excited to chat with an Agentic AI learner. What’s on your mind today? Are you exploring a particular AI topic, working on a project, or just curious about the latest developments? Let me know how I can help!'

In [16]:
## change the config that means changing the session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'I’m not sure—could you let me know what you’d like me to call you?'

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey my name is John")],
    config=config1
)
response.content

'Nice to meet you, John! How can I help you today?'

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'Your name is John.'

### Prompt templates
Prompt templates helps to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions. Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Anay")]})

AIMessage(content='Hello Anay! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'We need to respond politely, acknowledging the user, asking how can help.'}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 96, 'total_tokens': 134, 'completion_time': 0.040772774, 'completion_tokens_details': {'reasoning_tokens': 16}, 'prompt_time': 0.006389954, 'prompt_tokens_details': None, 'queue_time': 0.354236675, 'total_time': 0.047162728}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b6ac9-0bc0-7bc1-a0ce-0c54081f4a16-0', usage_metadata={'input_tokens': 96, 'output_tokens': 38, 'total_tokens': 134, 'output_token_details': {'reasoning': 16}})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Anay")],
    config=config
)
response.content

'Hi Anay! Nice to meet you. How can I help you today?'

In [25]:
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Anay")],"language":"Hindi"})
response.content

'नमस्ते अनाय! आपसे मिलकर खुशी हुई। मैं आपकी किस प्रकार सहायता कर सकता/सकती हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [27]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Anay")],'language':"Bengali"},
    config=config
)
response.content

'হ্যালো আনায়! কীভাবে আপনাকে সাহায্য করতে পারি?'

### Managing the conversation history

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messgaes you are passing in.

'trim_messages'  ->> helps to reduce how many messages we're sending to the model.

In [41]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages=[
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I am Anay"),
    AIMessage(content="HI!"),
    HumanMessage(content="I like butterscotch ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="Anytime"),
    HumanMessage(content="having fun?"),
    AIMessage(content="Hell yeah!"),
]
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like butterscotch ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2+2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Anytime', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hell yeah!', additional_kwargs={}, response_metadata={})]

In [42]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)

AIMessage(content='I’m not sure—what’s your favorite flavor? If you tell me a bit about what you like (sweet, fruity, chocolatey, creamy, adventurous, etc.), I can suggest something that might hit the spot!', additional_kwargs={'reasoning_content': 'User asks "What ice cream do i like". We don\'t know preferences. Should respond asking for preferences.'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 155, 'total_tokens': 231, 'completion_time': 0.083164902, 'completion_tokens_details': {'reasoning_tokens': 22}, 'prompt_time': 0.008741506, 'prompt_tokens_details': None, 'queue_time': 0.125816054, 'total_time': 0.091906408}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b6aee-7e3e-7262-9cce-58831d3bb435-0', usage_metadata={'input_tokens': 155, 'output_tokens': 76, 'total_tokens': 231, 'output_token_details': {'reason

In [43]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask for and answer it too")],
    "language":"English"
    }
)
response.content

'You asked for the simple arithmetic problem “2\u202f+\u202f2.”  \n**Answer:** 4.'

In [44]:
## Let's wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [45]:
response=with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="What's my name?")],
        "language":"english"
    },
    config=config
)
response.content

'I don’t have that information. If you’d like to share it, I’ll remember it for our conversation!'